In [1]:
from pathlib import Path

models_dir = Path("../../models/finetuning_studies/")

### Embedding Model

In [2]:
import sys
sys.path.append('./src')

In [3]:
from src.bertopic_finetuning import BERTopicTrainer

/mnt/NVMe2/VirtualEnvironments/general_python_venv/lib/python3.8/site-packages/cupy/_environment.py:445: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
2023-06-19 22:09:08.870949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorF

In [4]:
from bertopic_finetuning import BERTopicTrainer
from hyperparameters import BertopicHyperparameters
from util import initialize_logger, get_logfile_name, load_config

In [5]:
from pathlib import Path

config = load_config(config_file=Path("config/bertopic_finetuning_config_dataset_1.yml"))
logger = initialize_logger(logfile_name=config['logfile_name'], log_level=config['logging_level'])

2023-06-19 22:09:18,769 - INFO - util.py line: 42 - Logger initialized, Refer Logfile: /mnt/NVMe/workspace/github_projects/arxiv_dataset_insights/apps/bertopic_trainer/bertopic_trainer_logs.txt, LogLevel: 10


In [6]:
config

{'logging_dir': '../logs',
 'logfile_name': 'bertopic_trainer_logs.txt',
 'logging_level': 'DEBUG',
 'study_name': 'distilled_nli_models',
 'study_storage_filename': 'distilled_model_choice_experiments_dataset5.sql',
 'dataset_path': '../../dataset',
 'models_path': '../../models',
 'batch_size': 384,
 'dataset_index': 1,
 'n_trials': 5,
 'hyperparameters': {'model_name': ['sentence-transformers/distilroberta-base-paraphrase-v1',
   'sentence-transformers/stsb-distilroberta-base-v2',
   'sentence-transformers/distilbert-base-nli-stsb-quora-ranking',
   'distilbert-base-nli-mean-tokens'],
  'nr_topics': [5, 15, 25, 45, 75, 95, 135, 160, 190, 200, 210, 215, 300, 400],
  'top_n_words': [30, 60, 80, 100],
  'min_topic_size': [1, 5, 15, 30, 50, 100],
  'n_gram_range_start': [1],
  'n_gram_range_end': [1, 2, 3],
  'max_features': [500, 5000, 10000, 20000, 30000, 60000],
  'max_df': [0.75, 0.8, 0.86, 0.999],
  'min_df': [0.001, 0.01, 0.05, 0.1, 0.13],
  'lowercase': [True, False],
  'n_neighb

In [7]:
trainer = BERTopicTrainer(config=config, logger=logger)
trainer

In [9]:
import optuna
from optuna.trial._state import TrialState

distributions={
    'model_name': optuna.distributions.CategoricalDistribution([
        'sentence-transformers/distilroberta-base-paraphrase-v1',
        'sentence-transformers/stsb-distilroberta-base-v2',
        'sentence-transformers/distilbert-base-nli-stsb-quora-ranking',
        'distilbert-base-nli-mean-tokens'
    ]),

    'nr_topics': optuna.distributions.CategoricalDistribution([5, 15, 25, 45, 75, 95, 135, 160, 190, 200, 210, 215, 300, 400]),
    'top_n_words': optuna.distributions.CategoricalDistribution([30, 60, 80, 100]),
    'min_topic_size': optuna.distributions.CategoricalDistribution([1, 5, 15, 30, 50, 100]),
    'n_gram_range_start': optuna.distributions.CategoricalDistribution([1]),
    'n_gram_range_end': optuna.distributions.CategoricalDistribution([1, 2, 3]),
    'max_features': optuna.distributions.CategoricalDistribution([500, 5000, 10000, 20000, 30000, 60000]),
    'max_df': optuna.distributions.CategoricalDistribution([0.75, 0.8, 0.86, 0.999]),
    'min_df': optuna.distributions.CategoricalDistribution([0.001, 0.01, 0.05, 0.1, 0.13]),
    'lowercase': optuna.distributions.CategoricalDistribution([True, False]),
    'n_neighbors': optuna.distributions.CategoricalDistribution([3, 5, 7, 15, 25, 40]),
    'n_components': optuna.distributions.CategoricalDistribution([64, 128, 256]),
    'umap_metric': optuna.distributions.CategoricalDistribution(['euclidean']),
    'n_epochs': optuna.distributions.CategoricalDistribution([200]),
    'learning_rate': optuna.distributions.CategoricalDistribution([1.0]),
    'min_dist': optuna.distributions.CategoricalDistribution([0.1]),
    'random_state': optuna.distributions.CategoricalDistribution([65]),
    'min_cluster_size': optuna.distributions.CategoricalDistribution([1, 5, 15, 30, 50, 100]),
    'cluster_selection_epsilon': optuna.distributions.CategoricalDistribution([0.001, 0.01, 0.1, 0.2, 0.3, 0.4]),
    'hdbscan_metric': optuna.distributions.CategoricalDistribution(['euclidean']),
    'cluster_selection_method': optuna.distributions.CategoricalDistribution(['eom']),
    'topk': optuna.distributions.CategoricalDistribution([10])
}

params = dict(zip(
                    config['hyperparameters'].keys(),
                    [config['hyperparameters'][k][0] for k in config['hyperparameters'].keys()]
                ))

trial = optuna.create_trial(state=TrialState.RUNNING, params=params, distributions=distributions)

In [10]:
trainer.set_hyperparameters(trial)

2023-06-19 22:10:59,526 - DEBUG - hyperparameters.py line: 13 - Preparing hyperparameters
2023-06-19 22:10:59,528 - DEBUG - hyperparameters.py line: 78 - Finished preparing hyperparameters


In [26]:
trainer.datasets = {}

In [27]:
trainer.dataset_index = 5
# trainer.load_datasets(trainer.dataset_index)

load_splits = ['train', 'validation', 'test']

for split in load_splits:
    trainer.load_dataset(split=split, lemmatized='true')

Found cached dataset parquet (/mnt/NVMe/workspace/github_projects/arxiv_dataset_insights/apps/bertopic_trainer/../../dataset/cache_dir/parquet/default-85037356f516bd94/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/mnt/NVMe/workspace/github_projects/arxiv_dataset_insights/apps/bertopic_trainer/../../dataset/cache_dir/parquet/default-bab51ab8bcf387f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/mnt/NVMe/workspace/github_projects/arxiv_dataset_insights/apps/bertopic_trainer/../../dataset/cache_dir/parquet/default-54dfec133abc4bd4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
trainer.datasets

{'lemmatized_train': Dataset({
     features: ['title', 'categories', 'abstract', 'categories_list', 'label_counts', 'abstract_word_count', '__index_level_0__'],
     num_rows: 112830
 }),
 'lemmatized_validation': Dataset({
     features: ['title', 'categories', 'abstract', 'categories_list', 'label_counts', 'abstract_word_count', '__index_level_0__'],
     num_rows: 112830
 }),
 'lemmatized_test': Dataset({
     features: ['title', 'categories', 'abstract', 'categories_list', 'label_counts', 'abstract_word_count', '__index_level_0__'],
     num_rows: 112830
 })}

### Embeddings and Documents

In [29]:
train_embeddings = trainer.load_embeddings(split_name='train')
print(train_embeddings.shape)

validtion_embeddings = trainer.load_embeddings(split_name='validation')
print(validtion_embeddings.shape)

test_embeddings = trainer.load_embeddings(split_name='test')
print(test_embeddings.shape)

(112830, 768)
(112830, 768)
(112830, 768)


### Model

In [30]:
import os

os.listdir(models_dir)

['classifier_dataset3_finetuning.sql',
 'classifier_dataset3_study1',
 'classifier_dataset4_finetuning.sql',
 'classifier_dataset4_study1',
 'classifier_dataset4_study_training',
 'classifier_dataset5_finetuning.sql',
 'classifier_dataset5_study2',
 'dataset1_finetuning.sql',
 'dataset4_finetuning.sql',
 'dataset5_best_params_training',
 'dataset5_best_params_tuning4',
 'dataset5_finetuning.sql',
 'dataset5_partialfit.sql',
 'distilled_model_choice_experiments_dataset5.sql',
 'distilled_model_selection_sample_dataset5_3',
 'generate_embeddings.sql',
 'tuning_dataset1_study2',
 'tuning_dataset5_study1',
 'tuning_dataset5_study2',
 'tuning_dataset5_study3',
 'umap_and_hdbscan1_dataset5']

In [15]:
study_name = "distilled_model_selection_sample_dataset5_3"

from custom_bertopic import CustomBERTopic
model = CustomBERTopic.load(models_dir / study_name / 'best_model.bin')
model

### Calculate Coherence Score for all splits

In [31]:
for split in load_splits:
    print(f"Calculating coherence score for split: {split}")
    
    dataset = trainer.datasets[f'lemmatized_{split}']
    coherence_score = trainer.calculate_coherence_score(model=model, sentences=dataset['abstract'][:])
    print(f"Coherence Score for split: {split} is {coherence_score}")

Calculating coherence score for split: train


2023-06-19 22:37:35,677 - DEBUG - bertopic_finetuning.py line: 401 - Lemmatizing 6 topic words lists


Coherence Score for split: train is 0.5802270287769677
Calculating coherence score for split: validation


2023-06-19 22:38:12,478 - DEBUG - bertopic_finetuning.py line: 401 - Lemmatizing 6 topic words lists


Coherence Score for split: validation is 0.5678852871646215
Calculating coherence score for split: test


2023-06-19 22:38:45,999 - DEBUG - bertopic_finetuning.py line: 401 - Lemmatizing 6 topic words lists


Coherence Score for split: test is 0.5569436619576851


In [17]:
topics = model.get_topics()
len(topics)

7

In [34]:
model.get_topic_freq()

,Topic,Count
1,-1,75130
5,2,9062
7,0,8624
4,5,5805
0,4,5670
6,1,3236
2,3,3131
3,6,2172


In [35]:
model.get_topics()

{0: [('galaxies', 0.08708742456549733),
  ('star', 0.062114667626360656),
  ('galaxy', 0.05877394114192234),
  ('stellar', 0.05747142859859217),
  ('gas', 0.04439247709546281),
  ('radio', 0.04002352447181538),
  ('disk', 0.03815566780514373),
  ('galactic', 0.035049415269910504),
  ('accretion', 0.03067279223246181),
  ('dust', 0.030069266592327105),
  ('redshift', 0.02910525548160554),
  ('agn', 0.024788847050071377),
  ('population', 0.02256265648492371),
  ('flux', 0.02243336814746261),
  ('dwarf', 0.02223896971108184),
  ('solar', 0.019815211913599813),
  ('metallicity', 0.019611054133787012),
  ('halo', 0.01956146106129213),
  ('ngc', 0.01930640600116468),
  ('gammaray', 0.0189978635240227),
  ('planets', 0.017878494230020706),
  ('spectroscopic', 0.01706104295085566),
  ('disc', 0.016288466924685756),
  ('photometric', 0.016101616772882476),
  ('planet', 0.01595062136150338),
  ('kpc', 0.01581076500427297),
  ('kev', 0.015588458618221012),
  ('binaries', 0.015335608335839333),
 

In [ ]:
model.find_topics()

In [18]:
topics

{0: [('galaxies', 0.08708742456549733),
  ('star', 0.062114667626360656),
  ('galaxy', 0.05877394114192234),
  ('stellar', 0.05747142859859217),
  ('gas', 0.04439247709546281),
  ('radio', 0.04002352447181538),
  ('disk', 0.03815566780514373),
  ('galactic', 0.035049415269910504),
  ('accretion', 0.03067279223246181),
  ('dust', 0.030069266592327105),
  ('redshift', 0.02910525548160554),
  ('agn', 0.024788847050071377),
  ('population', 0.02256265648492371),
  ('flux', 0.02243336814746261),
  ('dwarf', 0.02223896971108184),
  ('solar', 0.019815211913599813),
  ('metallicity', 0.019611054133787012),
  ('halo', 0.01956146106129213),
  ('ngc', 0.01930640600116468),
  ('gammaray', 0.0189978635240227),
  ('planets', 0.017878494230020706),
  ('spectroscopic', 0.01706104295085566),
  ('disc', 0.016288466924685756),
  ('photometric', 0.016101616772882476),
  ('planet', 0.01595062136150338),
  ('kpc', 0.01581076500427297),
  ('kev', 0.015588458618221012),
  ('binaries', 0.015335608335839333),
 

In [ ]:
N  = 5000

In [19]:
transformed = model.transform(documents=documents[:N], embeddings=test_embeddings[:N])
transformed

NameError: name 'documents' is not defined

In [38]:
np.array(transformed).shape

(2, 5000)

In [20]:
transformed[1]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [34]:
model.hierarchical_topics(docs=transformed[0])

ValueError: All arrays must be of the same length

In [24]:
model.(documents=documents, embeddings=test_embeddings)

NameError: name 'embeddings' is not defined

In [35]:
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))["data"]